In [1]:
import common
import tqdm
import numpy as np
import pandas as pd

In [2]:
! ls /data/zhenyus/webcachesim/trace

memcachier0app100m.tr	   synthetic_urm.tr		 w10m_modify.tr.ant
memcachier0app100m.tr.ant  synthetic_urm.tr.ant		 w10m.tr
memcachier100m.tr	   _t_memcachier0app100m.tr	 w1k.tr
memcachier100m.tr.ant	   _t_memcachier0app100m.tr.ant  w1k.tr.ant
msr_proj_1_20m.tr	   _t_memcachier100m.tr		 w1m.tr
msr_proj_1_20m.tr.ant	   _t_memcachier100m.tr.ant	 w1m.tr.ant
msr_proj_1.tr		   w100k.tr			 w200m.mapping
msr_proj_1.tr.ant	   w100k.tr.ant			 w200m.tr
msr_proj_2.tr		   w10k.tr			 w200m.tr.ant
msr_proj_2.tr.ant	   w10m_modify.tr		 w50mBHR.tr


In [19]:
# timing scaning file: 1min 57s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each) for 200m
# %%timeit -n1 -r1
# with open(f'{common.WEBCACHESIM_ROOT}/trace/w200m.tr.ant') as f:
#     for line in f:
#         fields = line.split()

1min 57s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [29]:
# check duplicate size: for w200m.tr, no dup
# id_size = {}
# with open(f'{common.WEBCACHESIM_ROOT}/trace/w200m.tr.ant') as f:
#     for line in f:
#         fields = line.split()
#         t, idx, size, next_t = int(fields[0]), int(fields[1]), int(fields[2]), int(fields[3])
#         if idx in id_size and id_size[idx] != size:
#             print(1)
#         id_size[idx] = size

In [33]:
# [0, 99999] -> 0, [100000, 199999] -> 1
max_n_past_timestamps = 64
threshold = int(1e7)
# threshold = int(1e5)
# bin_width =  int(1e6)
# bin_width = 10000
# n_bin = threshold//bin_width

snapshot_interval = int(1e7)
# snapshot_interval = 100000

In [4]:
class Meta:
    def __init__(self, key, size, past_timestamp, future_timestamp):
        self._key = key;
        self._size = size;
        self._past_timestamp = past_timestamp;
        self._past_distances = [];
        self._future_timestamp = future_timestamp;
    
    def update(self, past_timestamp, future_timestamp):
        # distance
        distance = past_timestamp - self._past_timestamp
        self._past_distances.append(distance)
        if len(self._past_distances) > max_n_past_timestamps - 1:
            self._past_distances.pop(0)
        # timestamp
        self._past_timestamp = past_timestamp
        
        self._future_timestamp = future_timestamp

In [5]:
trace = []
with open(f'{common.WEBCACHESIM_ROOT}/trace/w200m.tr.ant') as f:
    for line in tqdm.tqdm(f): 
        fields = line.split()
        #t, key, size = 
        trace.append((int(fields[0]), int(fields[1]), int(fields[2]), int(fields[3])))

200000000it [13:55, 239379.58it/s]


In [6]:
len(trace)

200000000

In [34]:
snapshot_timestamps = np.arange(threshold, len(trace)-threshold+1, snapshot_interval, dtype=int)
snapshot_timestamps

array([ 10000000,  20000000,  30000000,  40000000,  50000000,  60000000,
        70000000,  80000000,  90000000, 100000000, 110000000, 120000000,
       130000000, 140000000, 150000000, 160000000, 170000000, 180000000,
       190000000])

In [35]:
start_ids = snapshot_timestamps - threshold
start_ids

array([        0,  10000000,  20000000,  30000000,  40000000,  50000000,
        60000000,  70000000,  80000000,  90000000, 100000000, 110000000,
       120000000, 130000000, 140000000, 150000000, 160000000, 170000000,
       180000000])

In [36]:
log_interval = threshold // 10

In [37]:
for start_idx in start_ids:    
    meta = {}  # key -> meta
    for request in trace[start_idx:start_idx+threshold]: 
        t, key, size, next_t = int(request[0]), int(request[1]), int(request[2]), int(request[3])
        if key not in meta:
            meta[key] = Meta(key, size, t, next_t)
        else:
            meta[key].update(t, next_t)
        if (t % log_interval) == 0:
            print(f't: {t}')

#     future = {}
#     for element in trace[t:t+threshold-1]:
#         _t, key = element[0], element[1]
#         if key not in future:
#             future[key] = [0] * 10
#         future[key][(_t-t)//bin_width] += 1
#         if (_t % bin_width) == 0:
#             print(f'collecting future at time {t}')
    
    snapshot = []
    for j, (k, m) in enumerate(meta.items()):
        if m._past_timestamp <= t - threshold:
            continue
        n_distances = len(m._past_distances)
        this_past_timestamp = m._past_timestamp
        if m._future_timestamp - t >= threshold:
            s = ['']
        else:
            s = [m._future_timestamp - t] 
        s.append(t - m._past_timestamp)
        for i in range(max_n_past_timestamps-1):
            # if exist
            if i >= len(m._past_distances):
                s.append('')
            else:
                this_past_timestamp -= m._past_distances[-(i+1)]
                if this_past_timestamp <= t - threshold:
                    s.append('')
                else:
                    s.append(m._past_distances[-(i+1)])
        snapshot.append(s)
        if (j % log_interval) == 0:
            print(f'j: {j}')
    with open(f'{common.WEBCACHESIM_ROOT}/snapshot_{t//snapshot_interval}.log', 'w') as f:
        for s in snapshot:
            f.write(','.join([str(i) for i in s]))
            f.write('\n')

t: 1000000
t: 2000000
t: 3000000
t: 4000000
t: 5000000
t: 6000000
t: 7000000
t: 8000000
t: 9000000
t: 10000000
j: 0
j: 1000000
j: 2000000
t: 11000000
t: 12000000
t: 13000000
t: 14000000
t: 15000000
t: 16000000
t: 17000000
t: 18000000
t: 19000000
t: 20000000
j: 0
j: 1000000
t: 21000000
t: 22000000
t: 23000000
t: 24000000
t: 25000000
t: 26000000
t: 27000000
t: 28000000
t: 29000000
t: 30000000
j: 0
j: 1000000
t: 31000000
t: 32000000
t: 33000000
t: 34000000
t: 35000000
t: 36000000
t: 37000000
t: 38000000
t: 39000000
t: 40000000
j: 0
j: 1000000
t: 41000000
t: 42000000
t: 43000000
t: 44000000
t: 45000000
t: 46000000
t: 47000000
t: 48000000
t: 49000000
t: 50000000
j: 0
j: 1000000
j: 2000000
t: 51000000
t: 52000000
t: 53000000
t: 54000000
t: 55000000
t: 56000000
t: 57000000
t: 58000000
t: 59000000
t: 60000000
j: 0
j: 1000000
t: 61000000
t: 62000000
t: 63000000
t: 64000000
t: 65000000
t: 66000000
t: 67000000
t: 68000000
t: 69000000
t: 70000000
j: 0
j: 1000000
t: 71000000
t: 72000000
t: 73000000

In [30]:
snapshot[0]

[36760,
 99999,
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ']

In [23]:
' '.join([str(i) for i in snapshot[1]])

'  99998                                                                                                                              '

In [29]:
' '.join([' ', ' ', ' '])

'     '

[[36760,
  99999,
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 [' ',
  99998,
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 [' ',
  99997,
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  '

In [271]:
m._future_timestamp

629192

In [267]:
meta[6355277154846406492]._future_timestamp - t

629191

In [199]:
! ls ../snapshot*

../snapshot_1.log  ../snapshot.log


In [205]:
fields = ['key'] + [f'e{i}' for i in range(10)] + ['i0'] + [f'd{i}' for i in range(1, max_n_past_timestamps)]
df_global = pd.read_csv(f'{common.WEBCACHESIM_ROOT}/snapshot_1.log', delimiter=' ', names=fields, nrows=100, skiprows=0)

In [208]:
df_global

,key,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9,i0,d1,d2,d3
0,6355277154846406492,1197,992,1817,831,1403,1625,672,1752,1293,971,280,190,27,84
1,5151997154538786157,1193,981,1881,835,1392,1619,638,1755,1256,946,703,159,425,164
2,8975157457366057399,0,0,0,0,0,0,0,0,0,0,439883,560114,1000000,1000000
3,1550438056448497221,0,0,0,1,0,0,1,0,0,0,847991,152005,1000000,1000000
4,681810876233799101,0,0,0,0,0,0,0,0,0,0,999995,1000000,1000000,1000000
5,8810281595397172408,0,0,1,0,0,0,0,0,0,0,999994,1000000,1000000,1000000
6,9528994978200312847,2,0,2,0,1,4,1,2,1,0,61228,21020,1168,1789
7,7541232769212342800,2,1,2,0,0,0,1,1,3,2,51387,8827,15098,113616
8,958924003583528526,0,0,1,0,0,0,0,0,0,0,851336,148655,1000000,1000000
9,6490982350858542199,0,0,0,0,0,0,0,0,0,0,380378,619612,1000000,1000000
